## Flood Vulnerability Index (FVI) Data Preparation
This notebook imports socioeconomic and physical data for FVI assessment.

In [347]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, QuantileTransformer
import fhv
from tabula import read_pdf
from functools import reduce

# UPAZILA SHAPEFILE
# ------------------------------------------------- #
shape = gpd.read_file('./data/admin_boundary/bgd_admbnda_adm3_bbs_20180410.shp')
# Convert ADM3_PCODE of Mymensingh (45) division (total 378 unions) (45 -> 30)
f45t30 = '30' + shape.loc[shape['ADM1_PCODE'] == '45', 'ADM3_PCODE'].str[2:]
shape.loc[shape['ADM1_PCODE'] == '45', 'ADM3_PCODE'] = f45t30.values
shape['ADM3_PCODE'] = shape['ADM3_PCODE'].astype(int)
# ------------------------------------------------- #
# shape[['ADM2_PCODE','ADM2_EN','ADM3_PCODE','ADM3_EN']].sort_values(
#     by='ADM3_PCODE').reset_index(drop=True).to_excel('./data/upazila_list.xlsx')

### Load demographic and socio-economic data
This section imports a variety of demographic and socio-economic data from multiple sources:
- [Bangladesh Bureau of Statistics (BBS)](http://203.112.218.65:8008/) 2011 census data downloaded from [BBS-REDATAM](http://203.112.218.69/binbgd/RpWebEngine.exe/Portal).
- [Bangladesh 2010 Poverty Maps (Zila Upazila)](http://203.112.218.65:8008/WebTestApplication/userfiles/Image/LatestReports/Bangladesh_ZilaUpazila_pov_est_2010.pdf) is obtained from [BBS Income, Expenditure & Poverty](http://203.112.218.65:8008/PageWebMenuContent.aspx?MenuKey=366).

In [342]:
census_table = [['PAGEWEAK','pos','Person','Demographic','Percent of weak population (age below 5 or above 65 years)','MinMax'],
                ['PFEMALE','pos','Person','Demographic','Percent of woman','MinMax'],
                ['PRURAL','pos','House','Built','Percent of households in rural areas','MinMax'], 
                ['PWEAKBUILT','pos','House','Built','Percent of households with weak materials','MinMax'],
                ['PNOWATER','pos','House','Built','Percent of households without public water supply','MinMax'],
                ['PNOSANITARY','pos','House','Built','Percent of households without sanitary facilities','MinMax'],
                ['PNOELEC','pos','House','Built','Percent of households without electricity','MinMax'],
                ['PDISABL','pos','Person','Social','Percent of population with any sort of disability','MinMax'],
                ['PLITERACY','pos','Person','Social','Percent of population who cannot read and write','MinMax'],
                ['PETHNIC','pos','Person','Social','Percent of ethnic population','MinMax'],
                ['PRENT','pos','House','Social','Percent of rented houses','MinMax'],
                ['PNOEMPLOY','pos','Person','Economic','Percent of population without employment','MinMax'],
                ['PAGRICULT','pos','Person','Economic','Percent of population with agricultural jobs','MinMax'],
                ['PPOOR','pos','House','Economic','Percentage of population below the upper poverty line','MinMax'],
                ['PPOOREXTR','pos','House','Economic','Percentage of population below the lower povery line','MinMax'],
                ['PNOPRIEDU','pos','Person','Education','Percent of population without primary education','MinMax'],
                ['PNOCOLLEGE','pos','Person','Education','Percent of population without college education','MinMax']
               ]
census_table = pd.DataFrame(census_table, columns=['Name','Sign','Type','Domain','Description','Normalization'])

In [343]:
# POPULATION DATA
# ------------------------------------------------- #
# BGD Census total population in 2011:  144,043,697
# BGD World Bank population in 2011:    149,273,778
# BGD World Bank population in 2017:    159,670,593
# ------------------------------------------------- #
df = fhv.LoadCensusBBS('./data/census2011/age 5 years group.xls')
popu2011 = df.sum(axis=1)
popu2017 = (popu2011/popu2011.sum()*159670593).astype(int)


# DEMOGRAPHIC
census = pd.DataFrame(index=df.index)
census.index.name = 'UID'
census['UID4'] = census.index % 10000   # Add a column of the last 4 digits of UID
# - PAGEWEAK: Percent weak population (age below 5 or above 65 years)
census['PAGEWEAK'] = df[df.columns[[0,14,15,16]]].sum(1)/df.sum(1)
# - PFEMALE: Percent of females
df = fhv.LoadCensusBBS('./data/census2011/sex.xls')
census['PFEMALE'] = df['Female']/df.sum(axis=1)


# BUILT ENVIRONMENT
# - PRURAL: Percent of households in rural areas
df = fhv.LoadCensusBBS('./data/census2011/Area of Residence.xls')
census['PRURAL'] = df['Rural']/df.sum(axis=1)
# - PWEAKBUILT: Percent of households with weak materials
# (#house_Kutcha_and_Jhupri / #house_total)
# *Pucca means high quality materials (e.g., cement or RCC)
# *Kutcha & Jhupri means weaker materials (e.g., mud, clay, lime, or thatched)
df = fhv.LoadCensusBBS('./data/census2011/Type of House.xls')
census['PWEAKBUILT'] = df[['Pucca','Semi-pucca']].sum(axis=1)/df.sum(1)
# - PNOWATER: Percent of households without public water supply
# *This includes "Other", excluding "Tap" and "Tube-well" water supply
df = fhv.LoadCensusBBS('./data/census2011/Source of Drinking Water.xls')
census['PNOWATER'] = df[df.columns[-1]]/df.sum(axis=1)
# - PNOSANITARY: Percent of households without sanitary facilities
# *This includes "Non-Sanitary" and "None" and excludes 
# *"Sanitary (with Water Seal)" and "Sanitary (no Water Seal)"
df = fhv.LoadCensusBBS('./data/census2011/Toilet Facilities.xls')
census['PNOSANITARY'] = df[df.columns[2:]].sum(axis=1)/df.sum(axis=1)
# - PNOELEC: Percent household without electricity
df = fhv.LoadCensusBBS('./data/census2011/Electricity Connection.xls')
census['PNOELEC'] = df['No']/df.sum(axis=1)


# SOCIAL
# - PDISABL: Percent of population with disability
# *This includes all kinds of disabilities (Speech, Vision, Hearing, Physical, Mental, Autistic)
df = fhv.LoadCensusBBS('./data/census2011/Disability.xls')
census['PDISABL'] = df[df.columns[1:]].sum(axis=1)/df.sum(axis=1)
# - PLITERACY: Percent of population who cannot read and write
df = fhv.LoadCensusBBS('./data/census2011/Literacy.xls')
census['PLITERACY'] = df['No']/df.sum(axis=1)
# - PETHNIC: Percent of ethnic population 
df = fhv.LoadCensusBBS('./data/census2011/Ethnic Population.xls')
census['PETHNIC'] = df['Yes']/df.sum(axis=1)
# - PRENT: Percent of rented houses
df = fhv.LoadCensusBBS('./data/census2011/Tenancy.xls')
census['PRENT'] = df[['Rented', 'Rent-free']].sum(axis=1)/df.sum(axis=1)


# EDUCATION
# - PNOPRIEDU: Percent of population who dont complete primary education
# *BGD's primary education is ClassI-ClassV
# *https://en.wikipedia.org/wiki/Education_in_Bangladesh#/media/File:BangEduSys.png
df = fhv.LoadCensusBBS('./data/census2011/Educational Attainment.xls')
census['PNOPRIEDU'] = df[df.columns[:5]].sum(axis=1)/df.sum(axis=1)
# - PNOCOLLEGE: Percent of population who don't attend college
census['PNOCOLLEGE'] = df[df.columns[:-4]].sum(axis=1)/df.sum(axis=1)


# EMPLOYMENT
# - PNOEMPLOY: Percent of population without employment
# *This includes "Employed" and "Household Work" and excludes "Looking For Job" and "Do Not Work"
df = fhv.LoadCensusBBS('./data/census2011/Activity Status.xls')
census['PNOEMPLOY'] = df[['Looking For Job','Do Not Work']].sum(axis=1)/df.sum(axis=1)
# - PAGRICULT : Percent of population with agricultural jobs
df = fhv.LoadCensusBBS('./data/census2011/Employment Field.xls')
census['PAGRICULT'] = df['Agriculture']/df.sum(axis=1)


# POVERTY
# Read PDF document and obtain data
df = read_pdf('./data/socioecon/Bangladesh_ZilaUpazila_pov_est_2010.pdf', 
             pages=list(range(3,13)), multiple_tables=False,
             pandas_options={'header': None, 'skiprows':2})
df.columns = ['zl-code','zila-name','UID4','upz-name','PPOOREXTR','PPOOR']
df = df.drop(['zl-code','zila-name','upz-name'], axis=1)
# Percentage to decimal
df[['PPOOREXTR','PPOOR']] = df[['PPOOREXTR','PPOOR']]/100
# Here we use only 4 upazila code to match with census UID, since all 4 digits are unique! Which means
assert len(np.unique(census.index % 10000)) == len(np.unique(df['UID4']))
# Sorting by UID4
df = df.set_index('UID4').sort_index()
# Merging
census = census.reset_index().merge(df, on='UID4').set_index('UID').drop('UID4',axis=1)
# Reordering to be matched with census_name
census = census[census_table['Name']]


# Additional columns
census_table['Scale'] = 'Upazila'
census_table['Source'] = 'BBS (2011)'

### Load variabes of Disaster-related Statistics (BBS, 2015)
- Original data (pdf file) is from [Bangladesh Disaster-related Statistics 2015: Climate Change and Natural Disaster Perspectives](http://203.112.218.65:8008/PageWebMenuContent.aspx?MenuKey=242).
- The data extraction is done by [LoadDisasterStatistics.ipynb](LoadDisasterStatistics.ipynb)

In [344]:
disaster_raw = pd.read_hdf('./data/disaster_statistics.hdf',key='data')
disaster_table = pd.read_hdf('./data/disaster_statistics_table.hdf', key='table')
# Expand District-level data to Upazila-level
disaster = pd.DataFrame(index=(census.index / 100).astype(int)).join(disaster_raw)
disaster.index = census.index

### Complete district-level indicators (total XX indicators)
- Merge DataFrames of Census and Disaster
- Flip signs of the indicators
    - Swap signs of the attributes expected to have a "negative" effect on vulnerability.
- Scaling values to 0-1 by Min/Max values

In [345]:
# Merge DataFrames of Census, Risk, and Impact
data_table = pd.concat([census_table, disaster_table]).reset_index(drop=True)
data_frames = [census,disaster]
data = reduce(lambda  left,right: pd.merge(left,right,on=['UID'],how='outer'), data_frames)

# Flip signs of the indicators
for index, row in data_table.iterrows():
    if row['Sign'] == 'neg':
        data[row['Name']] = -data[row['Name']].values
    elif row['Sign'] == 'pos':
        pass
    else:
        raise Exception("problem")
        
# Scaling to 0-1 with Max/Min values
scaler = MinMaxScaler()
data[data.columns] = scaler.fit_transform(data[data.columns])

In [346]:
# Re-order data
data_table.Description.values

array(['Percent of weak population (age below 5 or above 65 years)',
       'Percent of woman', 'Percent of households in rural areas',
       'Percent of households with weak materials',
       'Percent of households without public water supply',
       'Percent of households without sanitary facilities',
       'Percent of households without electricity',
       'Percent of population with any sort of disability',
       'Percent of population who cannot read and write',
       'Percent of ethnic population', 'Percent of rented houses',
       'Percent of population without employment',
       'Percent of population with agricultural jobs',
       'Percentage of population below the upper poverty line',
       'Percentage of population below the lower povery line',
       'Percent of population without primary education',
       'Percent of population without college education',
       'Percent of households affected by floods',
       'Percent of children did not attend to school du

### Load health data from geospatial data
- The main datasets are obtained from the [Directorate General of Health Services (DGHS)](https://dghs.gov.bd/index.php/en/home) - [Health Dashboard](http://103.247.238.81/webportal/pages/index.php).
- The data extraction is done by [LoadHealthData.ipynb](LoadHealthData.ipynb)


In [7]:
# Save data
if False:
    fn = './data/data.hdf'
    data.to_hdf(fn, 'data'); print('%s is saved.' % fn)
    fn = './data/data_table.hdf'
    data_table.to_hdf(fn, 'name'); print('%s is saved.' % fn)

./data/data.hdf is saved.
./data/data_table.hdf is saved.


## Load physical data